In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy

In [2]:
df=pd.read_csv("train.csv")

In [3]:
X =df.drop(['left','last_evaluation','satisfaction_level','average_montly_hours','time_spend_company','number_project','salary','sales'],axis=1)
X = pd.concat([X,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
y=df['left']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
orignalX=copy.deepcopy(X_train)
orignalY=copy.deepcopy(y_train)
testX=copy.deepcopy(X_test)
testY=copy.deepcopy(y_test)

In [5]:
df=pd.concat([X_train,y_train],axis=1)
df

,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,left
3633,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
5744,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
5791,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
9869,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
9826,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1
11141,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4302,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
329,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
8668,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
292,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [6]:
def findParentEntropy(df):
    if df.empty==True:
        return
    classCol=df.keys()[-1]
    #yeh sirf header return karega
    resultValues=df[classCol].unique()
    #all ouput values in output column
    entropy=0
    for value in resultValues:
        fraction=df[classCol].value_counts()[value]/len(df[classCol])
        entropy+=-fraction*np.log2(fraction+eps)
    return entropy
    

In [7]:
def findRelativeEntropy(df,attribute):
    if df.empty==True:
        return
    classCol= df.keys()[-1] 
    #last col assumed as result 
    #getting target class 
    resultValues=df[classCol].unique() 
    attributeNames=df[attribute].unique()
    entropy2 = 0
    for attr in attributeNames:
        entropy = 0
        for value in resultValues:
            #temp&hot with yes
            num = len(df[attribute][df[attribute]==attr][df[classCol]==value])
            #total hot
            den = len(df[attribute][df[attribute]==attr])
            fraction = num/(den+eps)
            entropy += -fraction*np.log2(fraction+eps)
        #entropy attribute/total*(entropy attribute_values)
        fraction2 = den/len(df)  
        entropy2 += -fraction2*entropy 
    return abs(entropy2)

In [8]:
def bestAttribute(df):
    if df.empty==True:
        return
    infoGain=[]
    for key in df.keys()[:-1]:
        infoGain.append(findParentEntropy(df)-findRelativeEntropy(df,key))
    return df.keys()[:-1][np.argmax(infoGain)]
    

In [9]:
class node:
    def __init__(self,value,left=None,right=None):
        self.value=value
        self.left=left
        self.right=right

In [10]:
def get_subtable(df, node,value):
    if df.empty==True:
        return
    return df[df[node] == value].reset_index(drop=True)


In [11]:
# def buildTree(df):
#     if len(df.columns)==1:
#         return
    
#     resultLabel= df.keys()[-1] 
#     attribute_name=bestAttribute(df)
#     #temprature
#     attValue = np.unique(df[attribute_name])
#     #hot,cold,windy
#     root=node(attribute_name)
# #     print(root.value)
#     for value in attValue:
#         subtable = get_subtable(df,attribute_name,value)
#         subtable =subtable.drop([attribute_name],axis=1)
#         clValue,counts = np.unique(subtable[resultLabel],return_counts=True) 
#         if len(counts)<=1:#Checking purity of subset
#             root.value=clValue[0]
# #             y.append(clValue)
#             return root
#         else :  
#                 if value==0:
#                     root.left=buildTree(subtable) #Calling the function recursively 
#                 else:
#                     root.right=buildTree(subtable)
       
                   
#     return root
def buildTree(df):
    if len(df.columns)==1:
        return
    
    resultLabel= df.keys()[-1] 
    clValue,counts = np.unique(df[resultLabel],return_counts=True)
    attribute_name=bestAttribute(df)
#     attValue = np.unique(df[attribute_name])
    root=node(attribute_name)
    
    if len(counts)<=1:
        if clValue[0]==0:
            root.negative=counts[0]
        else:
            root.positive=counts[0]
    else:
        root.positive=counts[1]
        root.negative=counts[0]
            
            
#         for value in attValue:
#             print(type(value))
        
        subtable0 = get_subtable(df,attribute_name,0)
        subtable0 =subtable0.drop([attribute_name],axis=1)
        subtable1 = get_subtable(df,attribute_name,1)
        subtable1 =subtable1.drop([attribute_name],axis=1)
        clValue,countleft = np.unique(subtable0[resultLabel],return_counts=True)
        clValue1,countright = np.unique(subtable1[resultLabel],return_counts=True)
        if(len(countleft)<=1):
            pass
        else:
            root.left=buildTree(subtable0) #Calling the function recursively
        if(len(countright)<=1):
            pass
        else:
            root.right=buildTree(subtable1)
       
                   
    return root

In [12]:
root=buildTree(df)
print(root.value)

Work_accident


In [13]:
print(root.right.value)
print(root.right.left)
print(root.left.value)
print(root.left.left.value)
print(root.left.right.value)


salary_high
salary_low
salary_high
sales_RandD


In [14]:
# def testing(row1,root):
#     if root.value==1 or root.value==0:
#         y1.append(root.value)
# #         print(root.value)
#         return
#     for i in row1.keys():
#         if i==root.value:
#             if row1[i]==0:
#                 testing(row1,root.left)
#             else:
#                 testing(row1,root.right)
def testing(row1,root):
#     if root==None:
#         return 
    #print(root.value)
    if root.left==None and root.right==None:
        if root.positive > root.negative:
            y1.append(1)
        else:
            y1.append(0)
        return
#     for i in row1.keys():
#         if i==root.value:
    i=root.value
    if row1[i]==0:
#         print("000")
        if root.left==None:
            if root.positive > root.negative:
                y1.append(1)
                return
            else:
                y1.append(0)
                return
        else:
            testing(row1,root.left)
    else:
#         print("111")
        if root.right==None:
            if root.positive > root.negative:
                y1.append(1)
                return
            else:
                y1.append(0)
                return 
        else:
            testing(row1,root.right)
                    
         

In [15]:
# def testingData(df):
#     for index,row in df.iterrows():
#         testing(row,root)

# y1=[]       
# testingData(X_test)  
# print(len(y1))
def testingData(df):
    for index,row in df.iterrows():
        testing(row,root)

y1=[] 
X_test
testingData(X_test)  
print(len(y1))



2248


In [16]:
print(confusion_matrix(y_test, y1))  
print(classification_report(y_test, y1)) 

[[1738    0]
 [ 510    0]]
              precision    recall  f1-score   support

           0       0.77      1.00      0.87      1738
           1       0.00      0.00      0.00       510

   micro avg       0.77      0.77      0.77      2248
   macro avg       0.39      0.50      0.44      2248
weighted avg       0.60      0.77      0.67      2248



/home/dhawal/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
accuracy=accuracy_score(y_test, y1)*100
print(accuracy)

77.31316725978647


In [18]:
classifier = DecisionTreeClassifier()
classifier.fit(orignalX, orignalY)  
y_pred = classifier.predict(testX)  
print(confusion_matrix(testY, y_pred))  
print(classification_report(testY, y_pred)) 
print(accuracy_score(testY,y_pred))

[[1738    0]
 [ 509    1]]
              precision    recall  f1-score   support

           0       0.77      1.00      0.87      1738
           1       1.00      0.00      0.00       510

   micro avg       0.77      0.77      0.77      2248
   macro avg       0.89      0.50      0.44      2248
weighted avg       0.82      0.77      0.68      2248

0.7735765124555161


In [19]:
accuracy=accuracy_score(y_test, y_pred)*100
print(accuracy)

77.35765124555161
